<a href="https://colab.research.google.com/github/joanglaunes/NUDFT/blob/main/NUDFT_KeOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script illustrates the use of KeOps for computing the multidimensional Non-Uniform Discrete Fourier Transform.
Computation is exact, brute-force and parallelized using GPU.

N.B. The first time the script is run, it will spend time compiling the required kernels. Re-run the script to get the correct timings.

In [ ]:
# installation of pykeops package
#!pip install pykeops[colab]
!pip install git+https://github.com/getkeops/keops.git#egg=pykeops[colab] --no-binary=pykeops

In [ ]:
import time
import math

Following Wikipedia :

The multidimensional NUDFT converts a 
$d$-dimensional array of complex numbers 
${\displaystyle x_{\mathbf {n} }}$ into another 
$d$-dimensional array of complex numbers 
${\displaystyle X_{\mathbf {k} }}$ defined by
$${\displaystyle X_{\mathbf {k} }=\sum _{\mathbf {n} =\mathbf {0} }^{\mathbf {N} -1}x_{\mathbf {n} }e^{-2\pi i\mathbf {p} _{\mathbf {n} }\cdot {\boldsymbol {f}}_{\mathbf {k} }}}$$
where 
${\displaystyle \mathbf {p} _{\mathbf {k} }\in [0,1]^{d}}$ are sample points, 
${\displaystyle {\boldsymbol {f}}_{\mathbf {n} }\in [0,N_{1}]\times [0,N_{2}]\times \cdots \times [0,N_{d}]}$ are frequencies, and 
${\displaystyle \mathbf {n} =(n_{1},n_{2},\ldots ,n_{d})}$ and 
${\displaystyle \mathbf {k} =(k_{1},k_{2},\ldots ,k_{d})}$ are 
$d$-dimensional vectors of indices from 0 to 
${\displaystyle \mathbf {N} -1=(N_{1}-1,N_{2}-1,\ldots ,N_{d}-1)}$

In [ ]:
def NUDFT(x,p,f):
  # Non-Uniform Discrete Fourier Transform (multidimensional)
  #   x : tensor of type torch.Tensor or numpy.ndarray and shape (N1,N2,...,ND), real or complex valued
  #   p, f : tensors of type torch.Tensor or numpy.ndarray and shapes (N1,N2,...,ND,D)
  s = x.shape
  x = x.reshape(1,-1,1)    # (N1,N2,...,ND) -> (1,N1xN2x...xND,1)
  p = p.reshape(1,-1,D)    # (N1,N2,...,ND,D) -> (1,N1xN2x...xND,D)
  f = f.reshape(-1,1,D)    # (N1,N2,...,ND,D) -> (N1xN2x...xND,1,D)
  f = -2*math.pi * f
  x_n, p_n, f_k = LazyTensor(x), LazyTensor(p), LazyTensor(f)
  M_nk = x_n * (p_n|f_k).exp1j()    # symbolic matrix of shape (N1xN2x...xND, N1xN2x...xND)
  return M_nk.sum(dim=0).reshape(s)

<br>
1) Testing with NumPy :

In [ ]:
import numpy as np
from pykeops.numpy import LazyTensor

dtype = np.float32

# specify N = (N1,N2,...,ND) 
N = 1000, 1000
D = len(N)

# define D-dimensional signal
x = np.random.randn(*N).astype(dtype)

# define D-dimensional array of sample points in [0,1]^D
p = np.random.randn(*N, D).astype(dtype)

# define D-dimensional array of frequencies in [0,N1]x[0,N2]x...x[0,ND]
f = np.random.randn(*N, D).astype(dtype)
for k in range(D):
  f[...,k] *= N[k]

In [ ]:
start = time.time()
X = NUDFT(x,p,f)
print("elapsed = ",time.time()-start)

elapsed =  2.9377636909484863


<br>
2) Testing with PyTorch, and computing gradient of $\|X\|^2$ wrt sample points $p_k$

In [ ]:
import torch
from pykeops.torch import LazyTensor

dtype = torch.float32
device_id = 'cuda'

# specify N = (N1,N2,...,ND) 
N = 1000, 1000
D = len(N)

# define D-dimensional signal
x = torch.randn(*N, dtype=dtype, device=device_id)

# define D-dimensional array of sample points in [0,1]^D
p = torch.rand(*N, D, dtype=dtype, requires_grad=True, device=device_id)

# define D-dimensional array of frequencies in [0,N1]x[0,N2]x...x[0,ND]
f = torch.rand(*N, D, dtype=dtype, device=device_id)
for k in range(D):
  f[...,k] *= N[k]

In [ ]:
start = time.time()
X = NUDFT(x,p,f)
print("elapsed = ",time.time()-start)

elapsed =  3.0278167724609375


In [ ]:
start = time.time()
G = torch.autograd.grad(torch.sum(X**2),p)[0]
print("elapsed = ",time.time()-start)

elapsed =  3.5868093967437744
